## Imports, Variable Initializations and Function Declarations

In [ ]:
# Import the necessary packages
%matplotlib widget
import rospy
import actionlib
import numpy as np
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import GridspecLayout, Layout
import matplotlib.pyplot as plt
from geometry_msgs.msg import Vector3
from assignment_2_2023.msg import PlanningAction, PlanningGoal, PlanningActionResult
from nav_msgs.msg import Odometry

# Declare the global variables
x_data = []
y_data = []
x_target = None
y_target = None

# Initialize the reached and cancelled target counts
reached_targets = 0
cancelled_targets = 0

# Variables to be used while plotting data
counter = 0
robot_active = False

# Initialize the action client 
target = PlanningGoal()
client = actionlib.SimpleActionClient('/reaching_goal', PlanningAction)

In [ ]:
# Declare the widgets to be used in the UI

# Declare the input textboxes
x_pos = widgets.FloatText(
    description='X_Position:',
    disabled=False
)
y_pos = widgets.FloatText(
    description='Y_Position:',
    disabled=False
)

# Declare the buttons
send_goal = widgets.Button(
    description='Send Goal',
    disabled=False,
    tooltip='Send the goal to gazebo',
)
cancel_goal = widgets.Button(
    description='Cancel Goal',
    disabled=False,
    tooltip='Cancel the latest goal sent to gazebo',
)

# Declare the output textbox
caption = widgets.Label(value='OUTPUT TERMINAL')
out = widgets.Output(layout= Layout(height='150px', border='1px solid black', overflow_y='scroll'))

In [ ]:
# Declare the callback function for the goal button
def on_goal_button_clicked(b):
    global x_data, y_data, robot_active
    x_data = []
    y_data = []
    ax[0].clear()
    ax[0].set_xlim(-10, 10)
    ax[0].set_ylim(-10, 10)
    ax[0].set_xlabel('X_position')
    ax[0].set_ylabel('Y_position')
    ax[0].set_title('Robot and Target Positions')
    robot_active = True
    update_target(x_pos.value, y_pos.value)
    target.target_pose.pose.position.x = x_pos.value
    target.target_pose.pose.position.y = y_pos.value
    client.send_goal(target)
    with out:
        print('[INFO] GOAL SET TO: ({0},{1})'.format(x_pos.value, y_pos.value))

In [ ]:
# Declare the callback function for the cancel button
def on_cancel_button_clicked(b):
    with out:
        print('[INFO] LATEST GOAL CANCELLED')
    client.cancel_goal()

In [ ]:
# Declare the functions to be used in plotting 

# Function to plot robot position
def plot(x_data, y_data):
    np_x_data = np.array(x_data)
    np_y_data = np.array(y_data)
    ax[0].plot(np_x_data, np_y_data)
    fig.canvas.draw()
    
# Function to plot the target position    
def update_target(x_target, y_target):
    if x_target is not None and y_target is not None:
        for line in ax[0].lines:
            if line.get_label() == 'Target':
                line.remove()
        ax[0].plot(x_target, y_target, 'gx', label='Target')
        ax[0].legend(loc='upper left')
        fig.canvas.draw()

# Function to plot the number of reached and cancelled targets
def plot_targets(reached_targets, cancelled_targets):
    targets = ['Target Reached', 'Target Cancelled']
    count = [reached_targets, cancelled_targets]
    color = ['green', 'red']
    ax[1].bar(targets, count, color=color)
    fig.canvas.draw()

In [ ]:
# Declare the callback function for the subscriber to /reaching/goal 
# that updates the reached or cancel count
def target_status_callback(msg):
    global robot_active, reached_targets, cancelled_targets
    target_status = msg.status.status
    if (target_status == 3):
        reached_targets = reached_targets + 1
        robot_active = False
    elif (target_status == 2):
        cancelled_targets = cancelled_targets + 1
        robot_active = False
    plot_targets(reached_targets, cancelled_targets)

In [ ]:
# Callback function for the subscriber to /odom
def odom_callback(msg):
    global counter
    counter = counter + 1
    x_data.append(round(msg.pose.pose.position.x,2))
    y_data.append(round(msg.pose.pose.position.y,2))
    if ((counter % 15 == 0) and (robot_active == True)):
        plot(x_data, y_data)

## User Interface

In [ ]:
# Initialize the rosnode 
rospy.init_node('UI_JUpyter')

# Declare the required subscribers
sub_odom = rospy.Subscriber('/odom', Odometry, odom_callback)
sub_goals = rospy.Subscriber('/reaching_goal/result', PlanningActionResult, target_status_callback)

# Declare the callbacks for the buttons
send_goal.on_click(on_goal_button_clicked)
cancel_goal.on_click(on_cancel_button_clicked)

# Specify the UI layout
grid = GridspecLayout(2, 2, width='800px')
grid[0, 0] = widgets.VBox([x_pos, y_pos])
grid[1, 0] = widgets.HBox([send_goal, cancel_goal])
grid[0:1, 1] = widgets.VBox([caption, out])
display(grid)

## Position and Target Plots

In [ ]:
# Declare the plots
fig, ax = plt.subplots(2, 1, figsize=(10,10))

# Plot for robot and target position
ax[0].set_xlim(-10, 10)
ax[0].set_ylim(-10, 10)
ax[0].set_xlabel('X_position')
ax[0].set_ylabel('Y_position')
ax[0].set_title('Robot and Target Positions')
np_x_data = np.array(x_data)
np_y_data = np.array(y_data)
ax[0].plot(np_x_data, np_y_data)

# Define a space between the plots
plt.subplots_adjust(hspace=0.3)

# Plot for reached and cancelled targets
targets = ['Target Reached', 'Target Cancelled']
count = [reached_targets, cancelled_targets]
color = ['green', 'red']
ax[1].set_ylim(0,20)
ax[1].bar(targets, count, color=color)
ax[1].set_ylabel('Count')
ax[1].set_title('Status of Targets')